In [20]:
REPO = "diligencefrozen/dcinside-data"
BRANCH = "main"

import sys, pathlib, os
ROOT = pathlib.Path("/content") / REPO.split("/")[-1]

if not ROOT.exists():
    !git clone -b {BRANCH} https://github.com/diligencefrozen/dcinside-data.git "{ROOT}"
%cd "{ROOT}"
!git pull

# src를 sys.path에 추가 (노트북이 어디서 열려도 동작)
cur = pathlib.Path.cwd()
for p in [cur, *cur.parents]:
    if (p/"src").exists():
        sys.path.insert(0, str(p/"src"))
        break

from paths import DATA, ZIP, MAIN, CLEAN, CLEAN_OKT
print("DATA =", DATA)


/content/dcinside-data
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (4/4), 285.44 KiB | 2.44 MiB/s, done.
From https://github.com/diligencefrozen/dcinside-data
   b6e7443..3f2da03  main       -> origin/main
Updating b6e7443..3f2da03
Fast-forward
 notebooks/03_tokenize.ipynb | 698 ++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 698 insertions(+)
 create mode 100644 notebooks/03_tokenize.ipynb
DATA = /content/dcinside-data/data


In [21]:
!wget -O "{ZIP}" https://codeload.github.com/diligencefrozen/dcinside-data/zip/refs/heads/main
!ls -lh "{ZIP}"


--2025-08-10 16:07:52--  https://codeload.github.com/diligencefrozen/dcinside-data/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/content/dcinside-data/data/dcinside-data-main.zip’

/content/dcinside-d     [         <=>        ]  54.12M  12.9MB/s    in 4.2s    

2025-08-10 16:07:57 (12.9 MB/s) - ‘/content/dcinside-data/data/dcinside-data-main.zip’ saved [56751052]

-rw-r--r-- 1 root root 55M Aug 10 16:07 /content/dcinside-data/data/dcinside-data-main.zip


In [22]:
# Colab/리눅스: 한글 폰트 자동 설치+세팅 (src/korean_setup.py 사용)
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd() / "src"))

from korean_setup import bootstrap_korean
print(bootstrap_korean())


{'font_path': '/usr/share/fonts/truetype/nanum/NanumGothic.ttf', 'family': 'NanumGothic', 'ok': True}


In [23]:
import sys, pathlib, pandas as pd
sys.path.append(str(pathlib.Path.cwd().parent / "src"))
from paths import CLEAN, CLEAN_OKT

# 토크나이저가 있으면 그걸 우선 사용
if CLEAN_OKT.exists():
    df = pd.read_parquet(CLEAN_OKT)
else:
    df = pd.read_csv(CLEAN)

# 여기서 일자/시간/작성자/토큰/도메인/상관 등 기술통계 실행
# (이미 만들어둔 함수 쓰거나, 간단히 바로 작성)
from collections import Counter
import re
from urllib.parse import urlparse

print("Rows:", len(df))

# 예: 시간 추이
df = df.copy()
df["datetime"] = pd.to_datetime(df.get("datetime"))
daily = (df.dropna(subset=["datetime"])
           .assign(date=lambda d: d["datetime"].dt.date)
           .groupby("date").size().reset_index(name="posts"))
display(daily.tail(30))

# 예: 토큰 탑
if "tokens" in df.columns:
    c = Counter()
    for ts in df["tokens"]:
        c.update(ts)
    top_tokens = pd.DataFrame(c.most_common(50), columns=["token","count"])
    display(top_tokens)


Rows: 204728


,date,posts
1545,2024-09-01,96
1546,2024-09-03,28
1547,2024-09-04,22
1548,2024-09-24,19
1549,2024-09-25,20
1550,2024-09-26,19
1551,2024-09-27,25
1552,2024-09-28,45
1553,2024-09-29,39
1554,2024-09-30,49


,token,count
0,',3373534
1,",",1531405
2,,1531405
3,나,241779
4,[,204728
5,],204728
6,안,195555
7,이,152056
8,사,150054
9,다,106872
